In [20]:
def rockfalls_import():
    read_in = open('C:/Users/dennis/Documents/STEIN_links/for_publication/model_outputs/stochastic_erosion/erosion_sets/import_dates.csv')
    myreader = csv.reader(read_in)
    rockfall_mat = np.empty((total_time, scenarios))
    
    for row in myreader:
        mat_gen_dates = row[:]
    
    rockfall_mat[:,:] = np.genfromtxt('C:/Users/dennis/Documents/STEIN_links/for_publication/model_outputs/stochastic_erosion/erosion_sets/rockfall_matrix_' + str(stoch_base_ER) + '_' + str(mat_gen_dates[-1]) + '.csv',
                                  delimiter = ',')
    ERs = np.genfromtxt('C:/Users/dennis/Documents/STEIN_links/for_publication/model_outputs/stochastic_erosion/erosion_sets/ER_list_' + str(stoch_base_ER) + '_' + str(mat_gen_dates[-1]) + '.csv',
                                  delimiter = ',')
        
    return rockfall_mat, ERs

In [7]:
def Be10fxn(dt, depth, conc0, scaling_factor):
    
    Be10_P0 = SLHL_Be10 * scaling_factor
    decay = 4.998e-7    
    B_Be = 0.026 # Granger and Smith 2000
    L3 = 4360 # g cm^-2 Granger and Smith 2000
    
    muons_10 = [0.026 * np.e**(-rock_rho * depth[i] / L3) for i in depth]
    spallogenic_10 = [Be10_P0 * np.e**(-mu * depth[i]) for i in depth]
    production_tot_10 = [muons_10[i] + spallogenic_10[i] for i in depth]
    
    conc10Be = [conc0[i] * np.e**(-decay) + (production_tot_10[i] / (decay + 0) * (1 - np.e**(-(decay + 0) * 1))) for i in depth]    

    return conc10Be

In [15]:
def C14fxn(dt, depth, conc0, scaling_factor):    
    C14_P0 = SLHL_C14 * scaling_factor   # production rate atoms / g / yr
    thalf = 5730    # half life 14-C, years
    decay = np.log(2) / thalf
    f_tot = 1.72*10**(-2) # Lupker et al. 

    mu = 2.62/160 # g * cm^-2
    A1 = 170.6 # Granger and Smith (2000)
    A2 = 36.75 # ibid.
    L1 = 788.6 # g cm^-2; Granger and Smith (2000)
    L2 = 2688 # g cm*-2; ibid.
    
    muons_14 = [f_tot * (A1*np.e**((-rock_rho * depth[i])/L1) + A2*np.e**((-2.62*depth[i])/L2)) for i in depth]  
    spallation_14 = [C14_P0 * np.e**(-mu * depth[i]) for i in depth]
    production_tot_14 = [spallation_14[i] + muons_14[i] for i in depth]
    
    conc14C = [conc0[i]*np.e**(-decay) + (production_tot_14[i] / (decay + 0) * (1 - np.e**(-(decay + 0) * 1))) for i in depth]

    return conc14C

In [6]:
def CRN_fxn(conc10Be, conc14C, rockfall_mat, dat_lin, year):
    fall_thickness = rockfall_mat[year] + 0.0001
    conc10Be = Be10fxn(dt, dat_lin, conc10Be, scaling_factor)
    conc14C = C14fxn(dt, dat_lin, conc14C, scaling_factor)

    depths = dat_lin[dat_lin > fall_thickness]
    minus = len(dat_lin) - len(depths)
    
    if minus > 0.0001:
        
        sec_Befxn = interp1d(x = (depths - fall_thickness), 
                             y = conc10Be[minus:],
                             kind = 'cubic',
                             fill_value = "extrapolate")

        conc10Be[:-minus] = sec_Befxn(x = dat_lin[:-minus])
        conc10Be[-minus:] = [0] * minus

    
        sec_Cfxn = interp1d(x = (depths - fall_thickness), y = conc14C[minus:], fill_value = "extrapolate")
        conc14C[:-minus] = sec_Cfxn(x = dat_lin[:-minus])
        conc14C[-minus:] = [0] * minus
    
    else:
        sec_Befxn = interp1d(x = depths, 
                             y = conc10Be[minus:],
                             kind = 'cubic',
                             fill_value = "extrapolate")

        conc10Be[:-minus] = sec_Befxn(x = dat_lin[:-minus])
        conc10Be[-minus:] = [0] * minus

    
        sec_Cfxn = interp1d(x = depths, y = conc14C[minus:], fill_value = "extrapolate")
        conc14C[:-minus] = sec_Cfxn(x = dat_lin[:-minus])
        conc14C[-minus:] = [0] * minus
        
    rockfall_mat = np.delete(rockfall_mat, (0), axis = 0)    
    
    return conc10Be, conc14C

In [6]:
def CRN_loop_fxn(total_time):
    dat_lin = np.arange(0,max_depth[1],1)
    all_rockfalls, ERs = rockfalls_import()
    all_mat = np.empty((total_time, 3, scenarios))
    
    for i in range(scenarios):
        conc14C = [0] * len(dat_lin)
        conc10Be = [0] * len(dat_lin)
        C_surf = np.empty((total_time,))
        Be_surf = np.empty((total_time,))
        C_tot = np.empty((max_depth[1],total_time))
        Be_tot = np.empty((max_depth[1],total_time))            
        export_mat = np.empty((total_time,3))
        
        year_falls = all_rockfalls[:,i]
        rockfall_plot = year_falls
        ER = ERs[i]
        
        for j in range(0,total_time):
            year = j
            conc10Be, conc14C = CRN_fxn(conc10Be, conc14C, year_falls, dat_lin, year)
            C_surf[j,] = np.average(conc14C[0:6])
            Be_surf[j,] = np.average(conc10Be[0:6])
            C_tot[:,j] = conc14C
            Be_tot[:,j] = conc10Be
            # print(year)
        
        print('Main C14 & Be10 loop scenario ' + str(i) + 'complete.')
        
        ratios = [(C_tot[0,i] / Be_tot[0,i]) for i in range(total_time)]

        export_mat[:,0] = C_surf
        export_mat[:,1] = Be_surf
        export_mat[:,2] = ratios
        all_mat[:,:,i] = export_mat
        
        C_export = C_tot
        Be_export = Be_tot
        
        if save_output == True:
            
            if save_only_surf == True:
                np.savetxt(r'C:/Users/dennis/Documents/STEIN_links/for_publication/model_outputs/stochastic_erosion/Be10_C14_simulation_outputs/STO_C14_Be10_ER' + str(ER) + '_expmat.csv',
                           export_mat,
                           delimiter = ',')

            else:
                np.savetxt(r'C:/Users/dennis/Documents/STEIN_links/for_publication/model_outputs/stochastic_erosion/Be10_C14_simulation_outputs/STO_C14_Be10_ER' + str(ER) + '_expmat.csv',
                       export_mat,
                       delimiter = ',')              
                np.savetxt(r'C:/Users/dennis/Documents/STEIN_links/for_publication/model_outputs/full_14C_10Be_data/Be10_C14_simulation_outputs/STO_C14_Be10_ER' + str(ER) + '_expC.csv',
                       C_export,
                       delimiter = ',')
                np.savetxt(r'C:/Users/dennis/Documents/STEIN_links/for_publication/model_outputs/full_14C_10Be_data/Be10_C14_simulation_outputs/STO_C14_Be10_ER' + str(ER) + '_expBe.csv',
                       Be_export,
                       delimiter = ',')
        
    return export_mat, all_mat